In [69]:
import numpy as np
import nltk
import matplotlib.pyplot as plt
import pandas as pd

#data scaling
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split , KFold, cross_val_score, LeaveOneOut
from sklearn.naive_bayes import MultinomialNB

import spacy

In [86]:
# preprocessing
data = pd.read_csv("TrainingSet1.csv")

print( data.head() )

#temporaire pour moins de données
#data = data.head()

intentions = data["intention"].unique()
data["intention"] = data["intention"].replace({"arrosage" : np.where( intentions == 'arrosage'),
                                              "soleil": np.where( intentions == 'soleil')})

                                       sentence intention
0  Est ce que j'ai besoin d'arroser ma plante ?  arrosage
1             Quand dois-je arroser ma plante ?  arrosage
2                   Dois-je arroser ma plante ?  arrosage
3           Comment puis-je arroser ma plante ?  arrosage
4        Ma plante a besoin de beaucoup d'eau ?  arrosage


In [87]:
y = list(data["intention"])

In [89]:
liste = data["sentence"].tolist()
j=0;

#dictionnaire des mots connus
words = set()

# process sentences
for i in data["sentence"]:
    # convert all letters to lower case
    i = i.lower()

    nlp_fr = spacy.load('fr_core_news_sm')
    tokens = nlp_fr(i)

    new_sentence = ''
    for token in tokens:
        if (token.pos_ == 'VERB' or (token.lemma_ != 'plante' and token.pos_ == 'NOUN')):
            new_sentence += str(token.lemma_) + ' '
            words.add(str(token.lemma_))
        
    liste[j] = new_sentence;
    j += 1

In [165]:
data["sentence"] = liste # ??

In [90]:
words = list(words)

In [91]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(liste).todense() #renvoie le bag of words

In [93]:
loo = LeaveOneOut()
loo.get_n_splits(X)

38

In [166]:
compt_naive_bayes = 0
compt_svm = 0
for train_index, test_index in loo.split(X):  #les differents jeux de données
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train = []
    y_train = []
    for index in train_index:
        X_train+=X[index].tolist()
        y_train.append(y[index])
    X_test = X[test_index].tolist()
    y_test = y[int(test_index)]
    
    clf_svm = svm.SVC(gamma ='auto')
    clf_bayes = MultinomialNB()
    clf_svm.fit(X_train, y_train)
    clf_bayes.fit(X_train, y_train)
    
    if int(clf_bayes.predict(X_test))==y_test:  # si prediction naive bayes correcte
        compt_naive_bayes += 1 
    if int(clf_svm.predict(X_test))==y_test:  # si prediction svm correcte
        compt_svm += 1 

In [167]:
print("Précision Naive Bayes :", compt_naive_bayes/len(X))
print("Précision SVM :", compt_svm/len(X))

Précision Naive Bayes : 0.9736842105263158
Précision SVM : 0.5263157894736842
